In [1]:
using LinearAlgebra, Test, Plots, Random, Distributions, Printf
pyplot()

nothing;

In [2]:
function strong_kron(A,B ::Array{T,N}) where {T<:Number,N}
    @assert size(A,3) == size(B,1)
    α=size(A,1)
    μ=size(A,2)
    β=size(A,3)
    ρ=size(B,2)
    γ=size(B,3)
    
    A = reshape(A,α*μ,β)
    B = reshape(B,β,ρ*γ)
    
    C = A*B
    
    C=reshape(C,α,μ*ρ,γ)
    
    return C
end
nothing;

In [3]:
function evaluate(S,n)
    N = length(S)
    A = copy(S[1])
    for j ∈ 2:N
        A=strong_kron(A,copy(S[j]))
    end
    A = reshape(A,n...)
    return A
end

evaluate (generic function with 1 method)

In [4]:
function TT_SVD(A, reqrank)
    
    N = ndims(A)
    S = copy(A)
    n = collect(size(A))
    Uk = []
   
    prev_dim = 1
    d = length(reqrank)
    
    ε = Vector{Float64}(undef, d-1)
    σ = Vector{Vector{Float64}}(undef, d-1)
    for k ∈ 1:d-1
       
        q = prod(n[k+1:N])
        B = reshape(S,prev_dim*n[k],q)
        
        Bk_svd = svd(B; full=false)
        
        U1 = Bk_svd.U[ :, 1:reqrank[k] ]                      #generating approximation
        V1t = Bk_svd.Vt[1:reqrank[k],: ]
        Sigma1 = Diagonal(Bk_svd.S[1:reqrank[k]])
        
        σ[k] = Bk_svd.S
        ρ = min(reqrank[k], length(σ[k]))
        ε[k] = norm(σ[k][ρ+1:end])
        
        U1 = reshape(U1,prev_dim,n[k],reqrank[k])
        S = Sigma1*V1t
        S = reshape(S,reqrank[k],q)
        
        push!(Uk,U1)
        prev_dim = reqrank[k]
    end
    U1 = reshape(S,prev_dim,n[d],1)
    push!(Uk,U1)
    return Uk, ε,σ
end

TT_SVD (generic function with 1 method)

In [5]:
function left_ortho(V)
    d = length(V)
    n = [size(V[i],2) for i in 1:d]
    r = [size(V[i],3) for i in 1:d]
    R = ones(1,1,1)
    for k = d : -1 :2
        V[k] = strong_kron(V[k],R)
        S = reshape(V[k],r[k-1],n[k]*r[k])
        LQ = lq(S)
        V[k] = reshape(LQ.Q[1:r[k-1],:],r[k-1],n[k],r[k])
        R = reshape(LQ.L,r[k-1],1,r[k-1])
    end
    V[1] = strong_kron(V[1],R)
    return V
end

left_ortho (generic function with 1 method)

In [6]:
function reapprox(U,reqrank)
    d = length(reqrank)
    n = [size(U[i],2) for i in 1:d]
    r = [size(U[i],3) for i in 1:d]
    prev_dim = 1
    L = ones(1,1,1)
    U_orth = left_ortho(copy(U))
    V = U_orth
    
    ε = Vector{Float64}(undef, d-1)
    σ = Vector{Vector{Float64}}(undef, d-1)
    for k=1:d-1
        V[k] = strong_kron(L,V[k])
        
        B = reshape(V[k], prev_dim*n[k],r[k])
        
        Bk_svd = svd(B,full=false)
        
        V[k] = Bk_svd.U[ :, 1:reqrank[k] ]  
        V1t = Bk_svd.Vt[1:reqrank[k],: ]
        Sigma1 = Diagonal(Bk_svd.S[1:reqrank[k]])
        
        σ[k] = Bk_svd.S
        ρ = min(reqrank[k], length(σ[k]))
        ε[k] = norm(σ[k][ρ+1:end])
        
        V[k] = reshape(V[k],prev_dim,n[k],reqrank[k])
        println(size(V[k]))
        L = Sigma1*V1t
        L = reshape(L,reqrank[k],1,r[k])
        
        prev_dim = reqrank[k]
    end
    V[d] = strong_kron(L,V[d])
    
    return V, ε,σ
end

reapprox (generic function with 1 method)

In [7]:
T = Float64
d=4
n = [20+i for i in 1:d]
r = [2*i for i in 1:d-1]

push!(r,1)
prev_dim = 1 
U_col = []
for i ∈ 1:d
    U = rand(Uniform(-1,1),prev_dim,n[i],r[i])/5
    #convert(Array{Float64}, U)
    push!(U_col,U)
    prev_dim = r[i]
end

In [8]:
println(n)
C = evaluate(U_col,n)


[21, 22, 23, 24]


21×22×23×24 Array{Float64, 4}:
[:, :, 1, 1] =
  0.00065434   -1.51104e-5    0.000339613  …   0.000362756  -0.000157466
 -9.42615e-6   -0.000858899  -0.000915622     -0.000291545  -0.00039915
 -0.000374331   0.000512197   0.000339522     -3.97032e-5    0.000325365
 -0.000223136   0.000286068   0.000181981     -3.00822e-5    0.000184954
  0.000701397  -0.000215641   0.00015261       0.000322374  -0.00026198
 -8.88813e-6    0.00014456    0.000148415  …   4.31821e-5    6.95883e-5
  0.000356665  -0.000982115  -0.000847272     -0.000126837  -0.000540872
 -0.00021924   -0.000109893  -0.000235651     -0.000159854  -9.52922e-7
 -0.0010326     0.000449942  -8.42374e-5      -0.000430448   0.000447585
  2.73001e-5   -0.000811677  -0.000845603     -0.000255164  -0.000385528
  3.98285e-6   -0.000176033  -0.000184444  …  -5.64282e-5   -8.31664e-5
  0.000599418  -0.000728699  -0.000446696      9.40663e-5   -0.000478263
 -9.16406e-5   -0.000403497  -0.000477545     -0.000185984  -0.000167468
 -0.000340

In [9]:
V = left_ortho(copy(U_col))
C_1 = evaluate(V,n)
println(norm(C-C_1,2)/norm(C,2))

5.991757329519925e-16


In [10]:
[size(V[i]) for i in 1:length(V)]

4-element Vector{Tuple{Int64, Int64, Int64}}:
 (1, 21, 2)
 (2, 22, 4)
 (4, 23, 6)
 (6, 24, 1)

In [11]:
r1 = r
r1[3]-=1

5

In [12]:
V1, norm_error, SVD_val = reapprox(U_col,r1)
C_2 = evaluate(V1,n)
println(norm(C-C_2,2)/norm(C,2))
[size(V1[i]) for i in 1:length(V1)]

(1, 21, 2)
(2, 22, 4)
(4, 23, 5)
0.26294430454762097


4-element Vector{Tuple{Int64, Int64, Int64}}:
 (1, 21, 2)
 (2, 22, 4)
 (4, 23, 5)
 (5, 24, 1)

In [13]:
V2 = U_col
V2[1] = V2[1].*3
C_3 = evaluate(V2,n)
println(norm(3*C-C_3))

3.588684932749564e-16


In [14]:
d=4
n1 = [5+i for i in 1:d]
rx = [2*i for i in 1:d-1]
ry = [i for i in 1:d-1]
push!(rx,1)
push!(ry,1)
prev_dimx = 1 
prev_dimy = 1
X_col = []
Y_col = []
for i ∈ 1:d
    X = rand((1:5),prev_dimx,n[i],rx[i])
    Y = rand((1:5),prev_dimy,n[i],ry[i])
    #convert(Array{Float64}, U)
    push!(X_col,X)
    push!(Y_col,Y)
    prev_dimx = rx[i]
    prev_dimy = ry[i]
end

In [15]:
X=evaluate(X_col,n)
Y=evaluate(Y_col,n)

Z_col = copy(X_col)
Z_col[1] = cat(X_col[1],Y_col[1],dims = 3)
d = length(X_col)
for i = 2:d-1
    Z_col[i]=cat(X_col[i],Y_col[i],dims=[1,3])
end
Z_col[d] = cat(X_col[d],Y_col[d],dims=1)
Z = evaluate(Z_col,n)
println(norm(X+Y-Z))

0.0


In [16]:
function tensor_add(X_col,Y,_col)
    Z_col = copy(X_col)
    Z_col[1] = cat(X_col[1],Y_col[1],dims = 3)
    d = length(X_col)
    for i = 2:d-1
        Z_col[i]=cat(X_col[i],Y_col[i],dims=[1,3])
    end
    Z_col[d] = cat(X_col[d],Y_col[d],dims=1)
    return Z
end

tensor_add (generic function with 1 method)

In [17]:
function mode_z_kronecker(X,Y)
    x1 = size(X,1)
    x2 = size(X,2)
    x3 = size(X,3)
    
    y1 = size(Y,1)
    y2 = size(Y,2)
    y3 = size(Y,3)
    
    X = permutedims(X, [1,3,2])
    Y = permutedims(Y, [1,3,2])
    
    X = reshape(X,x1*x3,x2)
    Y = reshape(Y,y1*y3,y2)
    
    Z = mapreduce(kron, hcat, eachcol(Y), eachcol(X))
    
    Z=reshape(Z,x1,x3,y1,y3,x2)
    Z = permutedims(Z,[1,3,2,4,5])
    Z = reshape(Z,x1*y1,x3*y3,x2)
    Z = permutedims(Z,[1,3,2])
    
    return Z
end

mode_z_kronecker (generic function with 1 method)

In [18]:
Z_col2 = copy(X_col)
for i = 1:d
    Z_col2[i] = mode_z_kronecker(copy(X_col[i]),copy(Y_col[i]))
end

In [19]:
X=evaluate(X_col,n)
Y=evaluate(Y_col,n)
Z2 = evaluate(Z_col2,n)
println(norm(X.*Y-Z2))

0.0


In [20]:
function hadamaz_product(X_col,Y_col)
    d = length(X_col)
    Z_col2 = copy(X_col)
    for i = 1:d
        Z_col2[i] = mode_z_kronecker(copy(X_col[i]),copy(Y_col[i]))
    end
    return Z_col2
end

hadamaz_product (generic function with 1 method)

In [48]:
function matrix_like_tensor(V)
    d = length(V)
    dimension = ones(2*d)
    m = [size(V[i],2) for i=1:d]
    n = [size(V[i],3) for i=1:d]
    dimension[1] = m[1]
    dimension[d+1] = n[1]
    A = V[1]
    for i = 2:d
        A_left = size(A,1)
        A_up = size(A,2)
        A_down = size(A,3)
        
        V_up = size(V[i],2)
        V_down = size(V[i],3)
        V_right = size(V[i],4)
        
        A = reshape(A,A_left*A_up*A_down,size(A,4))
        V[i] = reshape(V[i],size(V[i],1),V_up*V_down*V_right)
        
        A = A*V[i]
        
        A = reshape(A,A_left,A_up,A_down,V_up,V_down,V_right)
        A = permutedims(A,[1,2,4,3,5,6])
        A = reshape(A,A_left,A_up*V_up,A_down*V_down,V_right)
        
        println(A_left,' ',A_up,' ',A_down,' ',V_up,' ',V_down,' ',V_right)
        
        dimension[i]=m[i]
        dimension[i+d]=n[i]
    end
    A = reshape(A,Int64.(dimension)...)
    return A
end

matrix_like_tensor (generic function with 1 method)

In [49]:
d = 4
n = [2,3,4,2]
m = [1,2,3,4]
p = [1,1,1,1]
ru = [2,1,3,1]
rv = [2,3,2,1]
U1_col = []
V1_col = []
prev_dimu = 1
prev_dimv = 1
for i = 1:d
    U1=rand((1:5),prev_dimu,m[i],n[i],ru[i])
    V1=rand((1:5),prev_dimv,n[i],p[i],rv[i])
    push!(U1_col,U1)
    push!(V1_col,V1)
    prev_dimu = ru[i]
    prev_dimv = rv[i]
end

In [50]:
A = matrix_like_tensor(copy(U1_col))
B = matrix_like_tensor(copy(V1_col))
println(size(A))
println(size(B))
nothing;

1 1 2 2 3 1
1 2 6 3 4 3
1 6 24 4 2 1
1 2 1 3 1 3
1 6 1 4 1 2
1 24 1 2 1 1
(1, 2, 3, 4, 2, 3, 4, 2)
(2, 3, 4, 2, 1, 1, 1, 1)


In [51]:
W_col = []
X = copy(U1_col)
Y = copy(V1_col)

for i =1:d
    x_left = size(X[i],1)
    x_right = size(X[i],4)
    m_dim = size(X[i],2)
    mul_dim = size(X[i],3)
    
    p_dim = size(Y[i],3)
    y_left = size(Y[i],1)
    y_right = size(Y[i],4)
    
    X[i] = permutedims(X[i],[1,3,4,2])
    Y[i] = permutedims(Y[i],[2,1,3,4])

    X[i] = reshape(X[i],x_left*m_dim*x_right,mul_dim)
    
    Y[i] = reshape(Y[i],mul_dim,y_left*p_dim*y_right)
    
    W = X[i]*Y[i]
    W = reshape(W,x_left,m_dim,x_right,y_left,p_dim,y_right)
    W = permutedims(W,[1,4,2,5,3,6])
    W = reshape(W,x_left*y_left,m_dim,p_dim,x_right*y_right)
    
    push!(W_col,W)
end

In [52]:
W = matrix_like_tensor(copy(W_col))
println(size(W))

1 1 1 2 1 3
1 2 1 3 1 6
1 6 1 4 1 1
(1, 2, 3, 4, 1, 1, 1, 1)


In [53]:
big_m = prod(m)
big_n = prod(n)
big_p = prod(p)
println(big_m,' ',big_n,' ',big_p)

24 48 1


In [54]:
A = reshape(A,big_m,big_n)
B = reshape(B,big_n,big_p)
C = A*B
println(size(C))
W = reshape(W,big_m,big_p)
println(norm(C-W))

(24, 1)
4.339058732649733e7
